In [2]:
import urllib.parse
import rdflib as rdflib
import networkx as nx

In [7]:
import pandas as pd

In [45]:
def create_graph(g, nx_graph, matches_map):
    cnt = 0
    for s, p, o in g:
        if 'cites' in str(p).lower():
            s_name = str(urllib.parse.unquote(str(s).split('/')[-1].strip())).lower()
            o_name = str(urllib.parse.unquote(str(o).split('/')[-1].strip())).lower()
            if '#' in s_name and s_name.split('#')[-1] in matches_map:
                cnt += 1
                s_name = matches_map[s_name.split('#')[-1]].lower()
            if '#' in o_name and o_name.split('#')[-1] in matches_map:
                cnt += 1
                o_name = (matches_map[o_name.split('#')[-1]]).lower()
            nx_graph.add_edge(s_name, o_name)
    print(cnt)
    return nx_graph

In [44]:
def get_matchings():
    data = pd.read_csv('data/CORD_19/metadata.csv', usecols=['doi', 'sha'])
    match = dict()
    for _, row in data.iterrows():
        if not pd.isna(row['doi']) and not pd.isna(row['sha']):
            match[(str(row['sha']).strip()).lower()] = (str(row['doi']).strip()).lower()
    return match

In [3]:
g_kb = rdflib.Graph()
g_kb.parse('data/covid19_kb/kg.nt', format='nt')
generated_graph = nx.DiGraph()

http://dbpedia.org/datatype/polishZ\u0142oty does not look like a valid URI, trying to serialize this will break.


In [17]:
matches = get_matchings()

In [46]:
generated_graph = create_graph(g_kb, generated_graph, matches)

print('-------------Saving Graph--------------')
print('Number of nodes in the graph: ', len(generated_graph.nodes()))
print('Number of edges in the graph: ', len(generated_graph.edges()))
nx.write_gpickle(generated_graph, 'data/covid19_kb/references_network_whole.gpickle')
print('-------------Graph Saved--------------')

196842
-------------Saving Graph--------------
Number of nodes in the graph:  315790
Number of edges in the graph:  460264
-------------Graph Saved--------------
